# Factorización de Cholesky

#### Nombre: Benites Onofre Fernando Gabriel

In [93]:
import numpy as np
import math 
import random

### - Función que encuentre la factorización de Cholesky.

In [35]:
'''
Primero definimos nuestras clases de excepciones, las cuales
nos arrojarán los errores que se espera de capturen dado si 
la matríz ingresada no es real o positiva
'''

class PositiveMatrixCh(Exception): # clase para matríz A>0
    def __init__(self, n, message="Estás ingresando una Matríz que no es definida positiva, por lo tanto no hay factorización"):
        self.n = n
        self.message = message
        super().__init__(self.message)
        
class RealMatrixCh(Exception): # clase para matríz real A
    def __init__(self, n, message='Estás ingresando una Matríz que no es real, por lo tanto no hay factorización'):
        self.n = n
        self.message = message
        super().__init__(self.message)
        
class SymmetricMatrixCh(Exception):
    def __init__(self, n, message='Estás ingresando una Matríz que no es simétrica, por lo tanto no hay factorización'):
        self.n = n
        self.message = message
        super().__init__(self.message)

In [66]:
'''
Se espera que el formato de parámetro de A es 
[[a1,b1,...],...,[an,...,bn]] ya que se planea aplicarle 
la función np.matrix(A)
'''

def Cholesky(A):
    A_matrix = np.matrix(A) 
    x, y = A_matrix.shape
        
    if x == y:
        value, vector = np.linalg.eig(A_matrix)        
        if not np.all(value> 0) and np.all(A_matrix == A_matrix.T):
            raise PositiveMatrixCh(A_matrix)

    if not np.isreal(A_matrix).all():
        raise RealMatrixCh(A_matrix)
    
    if not np.all(A_matrix == A_matrix.T):
        raise SymmetricMatrixCh(A_matrix)

    L = np.zeros((x,y))
    
    # Diagonal
    for i in range(x):
        for k in range(i+1):
            sum_lk = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): 
                L[i][k] = math.sqrt(A[i][i] - sum_lk)
            else:
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - sum_lk))
    return L

Ahora capturemos las excepciones y mostremos una matríz que esté bien.

In [42]:
Cholesky([[-1,1],[2,1]])

SymmetricMatrixCh: Estás ingresando una Matríz que no es simétrica, por lo tanto no hay factorización

In [43]:
Cholesky([[1,1j],[2,1]])

RealMatrixCh: Estás ingresando una Matríz que no es real, por lo tanto no hay factorización

In [50]:
Cholesky([[1,2,3],[4,5,6]])

/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_732/380867406.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if not np.all(A_matrix == A_matrix.T):


SymmetricMatrixCh: Estás ingresando una Matríz que no es simétrica, por lo tanto no hay factorización

Ahora comprobemos que en efecto nuestro código nos da la factorización de Cholesky la cuál debe cumplir $$A=LL^T$$

In [67]:
Cholesky([[6, 15, 55],[15, 55, 225], [55, 225, 979]])@Cholesky([[6, 15, 55],[15, 55, 225], [55, 225, 979]]).T

array([[  6.,  15.,  55.],
       [ 15.,  55., 225.],
       [ 55., 225., 979.]])

### - Función que solucione el sistema Ax=b

Tenemos el sistema, $Ax = b$, de modo que tenemos: 
$$Ax = b$$ $$\Rightarrow LL^{T}x = b$$ entonces tenemos que la solución se vería como:
$$x = L^{-T}L^{-1}b$$

In [70]:
def solution_Cholesky(A,b):
    b = np.matrix(b)
    descompotition_Cholesky = Cholesky(A)
    trans_inv_desc_Cholesky = np.linalg.inv(descompotition_Cholesky.T)
    inv_desc_Cholesky = np.linalg.inv(descompotition_Cholesky)
    return trans_inv_desc_Cholesky@inv_desc_Cholesky@b

### - Pruebas del funcionamiento del programa

Ahora simulemos matrices simétricas, porque de no ser el caso y simplemente simular matríces, podríamos tener que tal matríz no es simétrica, lo cual nos arrojaría un error no propio de la función solución, de esta manera hacemos simulaciones poniendo un "umbral" de $E=0.009$

In [95]:
def Random_Symmetric_Matrix():
    n = random.randint(1,100)
    A = np.random.random_integers(-200,200,size=(n,n))
    A_symm = (A + A.T)/2
    b_matrix = []
    for i in range(n):
        b_matrix.append([random.randint(-70,70)])
    return A_symm, b_matrix

In [90]:
def test_solution_Cholesky(A,b):
    matrix_solution = solution_Cholesky(A,b)
    Ax_b = A@matrix_solution - b
    return np.linalg.norm(Ax_b)

In [99]:
def test_final(n):
    E = 0.009
    solution_test = 0

    for i in range(n):
        try: 
            A, b = Random_Symmetric_Matrix()
            solution_test = test_solution_Cholesky(A, b)
            if solution_test < E:
                continue
            else: 
                return f'''Encontramos que
                        no se cumple la desigualdad 
                        en el sistema Ax = b = {A}x = {b}'''
        except: 
            '''Al parecer alguno de estos sistemas no se pudo solucionar.'''
    else:
        return f'Nuestro programa funciona bien con {n} iteraciones'

In [98]:
n = 10000
test_final(n)

/var/folders/ln/r540h4b11mv526rjsf8_kdy40000gn/T/ipykernel_732/839567795.py:3: DeprecationWarning: This function is deprecated. Please call randint(-200, 200 + 1) instead
  A = np.random.random_integers(-200,200,size=(n,n))


'Nuestro programa funciona bien con 10000 iteraciones'